# 0. Import library and load data

In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import re
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (14, 6)

df = pd.read_csv("../data/raw/surat_uncleaned.csv")

# 1. Data Cross-Swap

Since there are many misclassified values inside the data, we need to cross-fill to get the suitable value within each row of the dataset. We gonna work with:
- ``transaction``: {``New Property``, ``Resale``}
- ``status``: {``Ready to Move``} or the value with the term "poss" inside. We gonna change it to Possession later
- ``floor``: It format is <string/int> out of <int>
- ``furnish``: {``Unfurnished``, ``Semi-Furnished``, ``Furnished``}
- ``facing``: value that contains the term of direction: "South", "North", "West", "East"

In [63]:
# transaction valid value
def is_valid_transaction(v):
    if pd.isna(v):
        return False
    return v in {"New Property", "Resale"}

In [64]:
# status valid value
def is_valid_status(v):
    if pd.isna(v):
        return False
    if not isinstance(v, str):
        return False
    return (
        v == "Ready to Move"
        or "Poss" in v
        or v == "Freehold"
    )

In [65]:
# floor valid value
def is_valid_floor(v):
    if pd.isna(v):
        return False
    if not isinstance(v, str):
        return False
    return "out of" in v

In [66]:
# furnishing valid value
def is_valid_furnishing(v):
    if pd.isna(v):
        return False
    return v in {"Unfurnished", "Semi-Furnished", "Furnished"}

In [67]:
# facing valid value
def is_valid_facing(v):
    if pd.isna(v):
        return False
    if not isinstance(v, str):
        return False
    return any(x in v for x in ["East", "West", "North", "South", "Main Road", "Garden/Park", "North - East", "North - West", "South - East", "South -West"])

In [68]:
valid_mapping = {
    "transaction": is_valid_transaction,
    "status": is_valid_status,
    "floor": is_valid_floor,
    "furnishing": is_valid_furnishing,
    "facing": is_valid_facing,
}

In [69]:
def fix_row(row, max_iter=20):
    row = row.copy()

    for _ in range(max_iter):
        changed = False

        for col, is_valid in valid_mapping.items():
            val = row[col]

            # We gonna skip if it is valid
            if pd.notna(val) and is_valid(val):
                continue

            # Find other valid value in other column
            for other_col, other_validator in valid_mapping.items():
                if other_col == col:
                    continue

                other_val = row[other_col]

                # assign the value of col
                if pd.notna(other_val) and is_valid(other_val):

                    # make sure other_col is good
                    if not other_validator(other_val):
                        # swap even if it is null
                        row[col], row[other_col] = other_val, val
                        changed = True
                        break

        if not changed:
            break

    return row

In [70]:
df = df.apply(fix_row, axis=1)

# 2. Handle non-numerical data

After we cross swapping the data, we dive deep down into cleaning every single columns inside the data

Firstly, we clean the ``areaWithType`` first since it is the most stable ones. We can see there are 4 main values inside the column corresponding to the area types: ``Super Area``, ``Carpet Area``, ``Plot Area``, ``Built Area`` and we need to remove the misclassified values. 

In [71]:
valid_areaWithType_value = ["Super Area", "Carpet Area", "Plot Area", "Built Area"]

df = df[df['areaWithType'].isin(valid_areaWithType_value)]

Back to ``transaction``, we first clean the column with the valid value ``New Property`` and ``Resale``. Any other value that out of this will be transfter to NaN

In [72]:
def analyze_value_distribution(col_name):
    print(f"\n=== Value Distribution for column: {col_name} ===")
    
    value_counts = df[col_name].value_counts(dropna=False)
    print("Number of unique values:", df[col_name].nunique())
    print("Top 10 most frequent values:\n", value_counts.head(10))

In [73]:
valid_transaction = {"New Property", "Resale"}

df["transaction"] = df["transaction"].where(
    df["transaction"].isin(valid_transaction),
    np.nan
)

In [74]:
analyze_value_distribution("transaction")


=== Value Distribution for column: transaction ===
Number of unique values: 2
Top 10 most frequent values:
 transaction
Resale          2719
New Property    1775
NaN               25
Name: count, dtype: int64


Inside column ``status``, we keep value ``Ready to Move`` and ``Freehold``. The value string that contains "Poss" string will be converted into ``Possession``

In [75]:
def normalize_status(v):
    if pd.isna(v) or not isinstance(v, str):
        return pd.NA

    if v == "Ready to Move":
        return "Ready to Move"

    if "Poss" in v:
        return "Possession"

    if v == "Freehold":
        return "Freehold"

    return pd.NA

In [76]:
df["status"] = df["status"].apply(normalize_status)

In [77]:
analyze_value_distribution("status")


=== Value Distribution for column: status ===
Number of unique values: 3
Top 10 most frequent values:
 status
Ready to Move    3078
Possession       1117
<NA>              210
Freehold          114
Name: count, dtype: int64


With ``floor`` we gonna keep the value with format "<string/int> out of <string/int>" and split them into two different columns that represents ``floor``(the current floor of that flats. If it is ground -> 0 and basement -> -1) and ``num_floor`` (the total floors of the building).

In [78]:
# Ensure string dtype
df["floor_raw"] = df["floor"].astype("string")

# Regex pattern
pattern = re.compile(
    r"^\s*(Ground|Basement|\d+)\s+out\s+of\s+(\d+)\s*$",
    re.IGNORECASE
)

def parse_floor(v):
    if pd.isna(v):
        return pd.NA, pd.NA

    m = pattern.match(v)
    if not m:
        return pd.NA, pd.NA

    current, total = m.groups()

    # Parse current floor
    current = current.lower()
    if current == "ground":
        floor_val = 0
    elif current == "basement":
        floor_val = -1
    else:
        floor_val = int(current)

    num_floor = int(total)

    return floor_val, num_floor

In [79]:
df[["floor", "num_floor"]] = df["floor_raw"].apply(
    lambda x: pd.Series(parse_floor(x))
)

df.drop(columns="floor_raw", inplace=True)

In [80]:
analyze_value_distribution("floor")


=== Value Distribution for column: floor ===
Number of unique values: 21
Top 10 most frequent values:
 floor
<NA>    848
5       564
0       386
7       376
6       365
1       364
4       304
3       300
2       286
8       221
Name: count, dtype: int64


In [81]:
analyze_value_distribution("num_floor")


=== Value Distribution for column: num_floor ===
Number of unique values: 23
Top 10 most frequent values:
 num_floor
<NA>    848
13      833
12      528
14      399
5       377
4       273
2       178
10      172
1       169
3       158
Name: count, dtype: int64


For ``furnishing``, we just keep the three main value ``Unfurnished``; ``Semi-Furnished`` and ``Furnished``. Everything left is NaN

In [82]:
valid_furnishing = {
    "Unfurnished",
    "Semi-Furnished",
    "Furnished"
}

df["furnishing"] = df["furnishing"].where(
    df["furnishing"].isin(valid_furnishing),
    pd.NA
)

In [83]:
analyze_value_distribution("furnishing")


=== Value Distribution for column: furnishing ===
Number of unique values: 3
Top 10 most frequent values:
 furnishing
Unfurnished       2739
<NA>               666
Semi-Furnished     585
Furnished          529
Name: count, dtype: int64


Now, we back to the most dirty column of ``facing`` where many misclassified values inside (1675/4525 = 37%) as well as ``description`` with 30.3% missing value percentage. We can check via code:

In [84]:
invalid_mask = ~df["facing"].apply(is_valid_facing)

num_invalid = invalid_mask.sum()

print(f"Number of invalid values in 'facing': {num_invalid}")

Number of invalid values in 'facing': 1675


Therefore, we gonna drop ``facing`` as well as ``description``

In [85]:
df = df.drop(columns=['facing', 'description'])

Finally, let's take a look at the dataframe after deciding to removing all NaN before moving to the next phases of cleaning the numerical dataset

In [86]:
cols = ["areaWithType", "transaction", "status", "furnishing", "floor", "num_floor"]

df = df.dropna(subset=cols)

In [87]:
df.describe().T

,count,unique,top,freq
property_name,3255,1412,3 BHK Apartment for Sale in Vesu Surat,70
areaWithType,3255,2,Super Area,2102
square_feet,3255,1073,1000 sqft,62
transaction,3255,2,Resale,1837
status,3255,3,Ready to Move,2337
floor,3255,21,5,539
furnishing,3255,3,Unfurnished,2313
price_per_sqft,3006,1615,"₹5,000 per sqft",59
price,3255,722,Call for Price,142
num_floor,3255,23,13,811


# 3. Handle numerical data

In [88]:
def parse_area_sqft(s):
    if pd.isna(s): 
        return np.nan
    t = str(s).strip().lower().replace(",", "")
    m = re.search(r"(\d+(\.\d+)?)", t)
    if not m:
        return np.nan
    num = float(m.group(1))

    if "sqft" in t:
        return num
    if "sqyd" in t or "sqyrd" in t:
        return num * 9
    if "sqm" in t or "sq.m" in t:
        return num * 10.7639
    if "acre" in t:
        return num * 43560
    if "rood" in t:
        return num * 10890
    if "ground" in t:
        return num * 2400

    return np.nan

def parse_ppsqft(s):
    if pd.isna(s): 
        return np.nan
    t = str(s).lower().replace("₹", "").replace(",", "")
    m = re.search(r"(\d+(\.\d+)?)", t)
    return float(m.group(1)) if m else np.nan

def parse_price_lakh(s):
    if pd.isna(s): 
        return np.nan
    t = str(s).strip().lower()
    if t == "call for price":
        return np.nan
    t = t.replace("₹", "").replace(",", "")
    m = re.search(r"(\d+(\.\d+)?)", t)
    if not m:
        return np.nan
    num = float(m.group(1))

    if "cr" in t:              # crore
        return num * 100
    if "lac" in t or "lakh" in t:
        return num

    return np.nan

def parse_bhk_from_name(s):
    if pd.isna(s):
        return np.nan
    m = re.search(r"(\d+)\s*bhk", str(s), flags=re.IGNORECASE)
    return float(m.group(1)) if m else np.nan

df["square_feet"]  = df["square_feet"].apply(parse_area_sqft)
df["price_per_sqft"]     = df["price_per_sqft"].apply(parse_ppsqft)
df["price_lakh"] = df["price"].apply(parse_price_lakh)
df["bhk"]        = df["property_name"].apply(parse_bhk_from_name)

df.drop(columns="price", inplace=True)
df = df.dropna(subset=["square_feet", "price_per_sqft", "price_lakh", "bhk"])
df.sample(6)

,property_name,areaWithType,square_feet,transaction,status,floor,furnishing,price_per_sqft,num_floor,price_lakh,bhk
4512,"5 BHK Apartment for Sale in Surya Heritage, Ve...",Super Area,7150.0,New Property,Ready to Move,11,Unfurnished,8503.0,12,608.0,5.0
3974,"4 BHK Apartment for Sale in Atlanta Eliza, Ves...",Carpet Area,2248.0,New Property,Ready to Move,10,Unfurnished,7791.0,20,323.0,4.0
769,2 BHK Apartment for Sale in ASHTVINAYAK RESIDE...,Carpet Area,650.0,Resale,Ready to Move,7,Unfurnished,3545.0,7,39.0,2.0
4309,"5 BHK Apartment for Sale in Avadh Ercole, Vesu...",Super Area,5260.0,Resale,Possession,8,Unfurnished,5703.0,12,300.0,5.0
3550,"3 BHK Apartment for Sale in Shaligram Flats, V...",Super Area,2200.0,New Property,Ready to Move,6,Unfurnished,5000.0,12,110.0,3.0
4095,4 BHK Apartment for Sale in Adajan Surat,Carpet Area,2238.0,New Property,Possession,9,Unfurnished,6100.0,13,248.0,4.0


In [89]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
square_feet,3003.0,3299.282735,76682.724029,3.0,1000.0,1400.0,2202.0,4.198405e+06
price_per_sqft,3003.0,5284.669331,7054.589076,4.0,3611.0,4500.0,5636.0,1.944440e+05
price_lakh,3003.0,133.329471,1015.195833,5.0,45.0,74.5,135.0,5.344400e+04
bhk,3003.0,2.856810,1.054230,1.0,2.0,3.0,3.0,1.000000e+01


# 4. Save purified data

In [90]:
# save cleaned data as CSV
df.to_csv("../data/processed/surat_cleaned.csv", index=False)
# save as numpy binary file
np.save("../data/processed/surat_cleaned.npy", df.to_records(index=False))